Importing Libraries needed to run script

In [2]:
import os
import numpy as np
import pandas as pd1
import shutil
import csv

In [5]:
verilog_modules = os.listdir("modules")
print(verilog_modules)

['top_andgate.v', 'top_dff.v', 'top_gray.v', 'top_muxcode.v']


In [6]:
top_mod = [i for i in verilog_modules if i[:3] == 'top'] 
print(top_mod)

['top_andgate.v', 'top_dff.v', 'top_gray.v', 'top_muxcode.v']


In [ ]:
os.system("vivado -mode batch -source tcl_create.tcl")

0

In [21]:
for j in top_mod:
    os.system("vivado -mode batch -source tcl_add.tcl -tclargs {}".format(j))


In [3]:
os.mkdir("results")

In [10]:
for k in top_mod:
    os.mkdir("results/"+k[:-2])
    os.system("vivado -mode batch -source tcl_run.tcl -tclargs {}".format(k))
    print("{} reported".format(k))

top_andgate.v reported
top_dff.v reported
top_gray.v reported
top_muxcode.v reported


Generating Result.csv file 
The approach is to search for certain text in the generated reports and extract out the numerical values needed.

In [11]:
top_files=os.listdir("results")
print(top_files)

['top_andgate', 'top_dff', 'top_gray', 'top_muxcode']


In [69]:
POWER=[]
DELAY=[]
LUT=[]

In [70]:
for i in top_files:
        with open("results/{}/{}".format(i,"power.txt"),"r") as File_PTU:
          
         words = File_PTU.read().split('\n')  
         for word in words:
             if 'Total On-Chip Power' in word:
                power=float(word.split()[6])
                POWER.append(power)
 
        with open("results/{}/{}".format(i,"timing.txt"),"r") as File_PTU:
          
         words = File_PTU.read().split('\n')  
         for word in words:
             if 'Data Path Delay' in word:
                delay=float(word.split()[3][:-2])
                DELAY.append(delay)
 
        with open("results/{}/{}".format(i,"utilization.txt"),"r") as File_PTU:
          
         words = File_PTU.read().split('\n')  
         for word in words:
             if 'Slice LUTs' in word:
                luts=float(word.split()[4])
                LUT.append(luts)   
print(LUT)    
print(DELAY)  
print(POWER) 

              
            





[1.0, 0.0, 2.0, 2.0]
[5.624, 4.014, 6.257, 6.22]
[0.371, 0.396, 1.674, 1.03]


In [71]:
ROW=["POWER","DELAY","LUTs"]
display=pd1.DataFrame([POWER,DELAY,LUT],index=ROW)
display.columns=top_files

display.to_csv("RESULT_CSV.csv",index=ROW)
print(display)

       top_andgate  top_dff  top_gray  top_muxcode
POWER        0.371    0.396     1.674         1.03
DELAY        5.624    4.014     6.257         6.22
LUTs         1.000    0.000     2.000         2.00
